In [29]:
# Load Library

packages <- c("dplyr", "haven","ggplot2")

install_if_missing <- function(pkg) {
  if (!requireNamespace(pkg, quietly = TRUE)) {
    install.packages(pkg)
  }
  library(pkg, character.only = TRUE)
}

lapply(packages, install_if_missing)

#test


[[1]]
 [1] "ggplot2"   "haven"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[2]]
 [1] "ggplot2"   "haven"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"     

[[3]]
 [1] "ggplot2"   "haven"     "dplyr"     "stats"     "graphics"  "grDevices"
 [7] "utils"     "datasets"  "methods"   "base"

# Read Data

## Selected Variables

In [49]:
var_tot <- readLines("Data/TOT_Variables.txt") # Selected Variable for Diet
length(var_tot)

var_nhanes <- readLines("Data/Nhanes_Variables.txt") # Selected Variable for Nhanes
length(var_nhanes)

[1] 121

[1] 10

## Read Nhanes Data

In [30]:
ds <- read.csv("Data/nhanes_data.csv")
head(ds)


,X,svy_id,svy_weight_mec,svy_psu,svy_strata,svy_year,svy_subpop_htn,svy_subpop_chol,demo_age_cat,demo_race,⋯,cc_egfr,cc_hba1c,cc_egfr_lt60,cc_acr_gteq30,cc_cvd_mi,cc_cvd_chd,cc_cvd_stroke,cc_cvd_ascvd,cc_cvd_hf,cc_cvd_any
,<int>,<int>,<dbl>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,⋯,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,1,12,95494.214,2,6,1999-2000,1,1,18 to 44,Non-Hispanic White,⋯,83.19409,5.2,No,Yes,No,No,No,No,No,No
2,2,21,2910.630,2,3,1999-2000,1,1,18 to 44,Hispanic,⋯,117.88244,5.0,No,No,NA,NA,NA,No,NA,No
3,3,27,1935.098,2,13,1999-2000,1,0,18 to 44,Hispanic,⋯,NA,NA,NA,No,NA,NA,NA,No,NA,No
4,4,28,41107.191,2,1,1999-2000,1,1,18 to 44,Non-Hispanic White,⋯,139.84511,4.8,No,No,NA,NA,NA,No,NA,No
5,5,56,89768.526,1,12,1999-2000,1,0,18 to 44,Non-Hispanic White,⋯,126.51034,5.5,No,No,No,No,No,No,No,No
6,6,57,102609.750,2,5,1999-2000,1,1,18 to 44,Non-Hispanic White,⋯,91.68801,4.9,No,No,No,No,No,No,No,No


## Read DT1TOT Data

In [48]:
## DR1TOT_A Have different wate variable name, rename it
tot_a <- read_xpt("Data/DR1TOT_A.xpt")
names(tot_a)[names(tot_a) == "DRXTWATE"] <- "DRXTMOIS"
tot_a <- tot_a %>% 
            select(var_tot)
head(tot_a)

Warning message:
"Using an external vector in selections was deprecated in tidyselect 1.1.0.
ℹ Please use `all_of()` or `any_of()` instead.
  # Was:
  data %>% select(var_tot)

  # Now:
  data %>% select(all_of(var_tot))

See <https://tidyselect.r-lib.org/reference/faq-external-vector.html>."


SEQN,DRXTPROT,DRXTCARB,DRXTTFAT,DRXTSFAT,DRXTMFAT,DRXTPFAT,DRXTCHOL,DRXTFIBE,DRXTVAIU,⋯,DRQ370QQ,DRD370R,DRQ370RQ,DRD370S,DRQ370SQ,DRD370T,DRQ370TQ,DRD370U,DRQ370UQ,DRD370V
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,31.96,250.36,27.24,9.40,9.00,6.40,46.55,7.41,2797.65,⋯,NA,2,NA,2,NA,2,NA,2,NA,2
2,123.16,350.37,71.95,24.34,26.48,11.94,313.95,36.99,4323.62,⋯,NA,2,NA,2,NA,2,NA,2,NA,2
3,40.19,233.63,49.94,15.61,19.57,11.84,86.22,11.16,5754.31,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,56.16,191.03,56.20,27.54,20.13,4.67,194.78,5.45,1426.17,⋯,NA,2,NA,2,NA,2,NA,2,NA,2
5,97.13,253.98,114.52,36.00,60.55,11.35,180.57,17.28,10292.63,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,24.63,202.11,25.15,7.78,7.84,7.15,38.45,6.24,1394.76,⋯,NA,2,NA,2,NA,2,NA,2,NA,2


In [53]:
folder_path <- "Data/" 
file_names <- paste0("DR1TOT_", LETTERS[2:10], ".xpt")  

ds_diet <- tot_a 

for (file in file_names) {
    file_path <- file.path(folder_path, file)  # Construct full file path
    if (file.exists(file_path)) {  # Ensure the file exists before reading
        temp_data <- read_xpt(file_path)  %>%  # Read the file
                        select(any_of(var_tot)) # Select Variables
        
        ds_diet <- bind_rows(ds_diet, temp_data)  # Append data
    } else {
        message(paste("File not found:", file))  # Notify if a file is missing
    }
}

head(ds_diet)


SEQN,DRXTPROT,DRXTCARB,DRXTTFAT,DRXTSFAT,DRXTMFAT,DRXTPFAT,DRXTCHOL,DRXTFIBE,DRXTVAIU,⋯,DRQ370QQ,DRD370R,DRQ370RQ,DRD370S,DRQ370SQ,DRD370T,DRQ370TQ,DRD370U,DRQ370UQ,DRD370V
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,31.96,250.36,27.24,9.40,9.00,6.40,46.55,7.41,2797.65,⋯,NA,2,NA,2,NA,2,NA,2,NA,2
2,123.16,350.37,71.95,24.34,26.48,11.94,313.95,36.99,4323.62,⋯,NA,2,NA,2,NA,2,NA,2,NA,2
3,40.19,233.63,49.94,15.61,19.57,11.84,86.22,11.16,5754.31,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
4,56.16,191.03,56.20,27.54,20.13,4.67,194.78,5.45,1426.17,⋯,NA,2,NA,2,NA,2,NA,2,NA,2
5,97.13,253.98,114.52,36.00,60.55,11.35,180.57,17.28,10292.63,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
6,24.63,202.11,25.15,7.78,7.84,7.15,38.45,6.24,1394.76,⋯,NA,2,NA,2,NA,2,NA,2,NA,2
